In [1]:
# считывает MovieLens
from os import path
import pandas as pd

data_dir = "./ml-latest-small"

def read_csv(filename: str):
    data = pd.read_csv(path.join(data_dir, filename + ".csv"))
    return data

ratings = read_csv("ratings")
movies = read_csv("movies")
tags = read_csv("tags")

In [21]:
tags.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1296 entries, 0 to 1295
Data columns (total 6 columns):
userId       1296 non-null int64
movieId      1296 non-null int64
tag          1296 non-null object
timestamp    1296 non-null int64
movie_id     1296 non-null int16
tag_id       1296 non-null int16
dtypes: int16(2), int64(3), object(1)
memory usage: 45.6+ KB


In [8]:
tags["movie_id"] = tags["movieId"].astype("category").cat.codes.copy()
tags["tag_id"] = tags["tag"].astype("category").cat.codes.copy()

In [10]:
tags.head()

,userId,movieId,tag,timestamp,movie_id,tag_id
0,15,339,sandra 'boring' bullock,1138537770,22,464
1,15,1955,dentist,1193435061,104,252
2,15,7478,Cambodia,1170560997,301,22
3,15,32892,Russian,1170626366,353,121
4,15,34162,forgettable,1141391765,355,296


In [11]:
# функция, которая красиво печатает информацию о разреженных матрицах
from scipy.sparse import csr_matrix

def sparse_info(sparse_matrix: csr_matrix) -> None:
    print("Размерности матрицы: {}".format(sparse_matrix.shape))
    print("Ненулевых элементов в матрице: {}".format(sparse_matrix.nnz))
    print("Доля ненулевых элементов: {}"
          .format(sparse_matrix.nnz / sparse_matrix.shape[0] / sparse_matrix.shape[1])
    )
    print("Среднее значение ненулевых элементов: {}".format(sparse_matrix.data.mean()))
    print("Максимальное значение ненулевых элементов: {}".format(sparse_matrix.data.max()))
    print("Минимальное значение ненулевых элементов: {}".format(sparse_matrix.data.min()))

In [12]:
from scipy.sparse import csr_matrix
import numpy as np


In [16]:
data = []
for i in tags["movie_id"]:
    data.append(1)
last_movie_id = len(tags["movie_id"])
last_tag_id = len(tags["tag_id"])

In [20]:
# Создаем матрицу  movies tags#
#from scipy.sparse import coo_matrix
#import numpy as np
#movie_tag_matrix = coo_matrix((
#    data,
#    (
#        mov_tag["movie_id"],
#        mov_tag["tag_id"]
#    )
#))

#sparse_info(movie_tag_matrix)

movie_x_tag = tags[["movie_id", "tag_id"]].as_matrix()
mean_rating = ratings["rating"].mean()
movie_tag_matrix = csr_matrix(
    (
        data,
        (
            [pair[0] for pair in movie_x_tag],
            [pair[1] for pair in movie_x_tag],
        )
    ),
    shape=(last_movie_id + 1, last_tag_id + 1),
    dtype=np.float32
)

sparse_info(movie_tag_matrix)

Размерности матрицы: (1297, 1297)
Ненулевых элементов в матрице: 1272
Доля ненулевых элементов: 0.000756148611736116
Среднее значение ненулевых элементов: 1.0188679695129395
Максимальное значение ненулевых элементов: 4.0
Минимальное значение ненулевых элементов: 1.0


In [22]:
import implicit

In [23]:
#  Строим матрицу схожести по тегам и фильмам.
import time
from implicit.nearest_neighbours import CosineRecommender

model_movie_tag = CosineRecommender()
print("строим матрицу схожести по косинусной мере")
start = time.time()
model_movie_tag.fit(movie_tag_matrix)
print("построили матрицу схожести по косинусной мере за {} секунд".format(
        time.time() - start))
sparse_info(model_movie_tag.similarity)

строим матрицу схожести по косинусной мере
построили матрицу схожести по косинусной мере за 0.012993335723876953 секунд
Размерности матрицы: (1297, 1297)
Ненулевых элементов в матрице: 8367
Доля ненулевых элементов: 0.004973817165405726
Среднее значение ненулевых элементов: 0.8192996221759413
Максимальное значение ненулевых элементов: 1.0000000000000002
Минимальное значение ненулевых элементов: 0.047457899787624956


In [32]:
# Получаем персональные рекомендации для пользователей из обучающего и тестого сетов по kNN и матрице схожести по тегам.
output_file = open("pred.txt", "w")
import implicit
print("получаем рекомендации для всех пользователей")
start = time.time()
movie_tag_T_csr = movie_tag_matrix.T.tocsr()

for movie_id in tags["movie_id"].unique():
    for movieId, score in model_movie_tag.recommend(movie_id, movie_tag_T_csr):
        output_file.write("%s\t%s\t%s\n" % (userId, movieId, score))
print("получили рекомендации для всех пользователей за {} секнуд".format(
        time.time() - start))

получаем рекомендации для всех пользователей
получили рекомендации для всех пользователей за 0.41100406646728516 секнуд


In [30]:
 model_movie_tag.recommend(1, movie_tag_T_csr)

[(145, 0.40824829046386307),
 (116, 0.23570226039551589),
 (617, 0.20412414523193154),
 (279, 0.16666666666666671)]

In [33]:
from lightfm import LightFM
from lightfm.evaluation import precision_at_k, reciprocal_rank, recall_at_k, auc_score

c:\projects\lightfm\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn('LightFM was compiled without OpenMP support. '


In [34]:
# перекодируем id с пробелами в плотные
ratings["movie_id"] = ratings["movieId"].astype("category").cat.codes.copy()
ratings["user_id"] = ratings["userId"].astype("category").cat.codes.copy()
last_movie_id = ratings["movie_id"].max()
last_user_id = ratings["user_id"].max()

In [35]:
user_x_item = ratings[["user_id", "movie_id"]].as_matrix()
mean_rating = ratings["rating"].mean()
user_item_matrix = csr_matrix(
    (
        (ratings["rating"] > mean_rating).tolist(),
        (
            [pair[0] for pair in user_x_item],
            [pair[1] for pair in user_x_item],
        )
    ),
    shape=(last_user_id + 1, last_movie_id + 1),
    dtype=np.float32
)

sparse_info(user_item_matrix)

Размерности матрицы: (671, 9066)
Ненулевых элементов в матрице: 100004
Доля ненулевых элементов: 0.016439141608663475
Среднее значение ненулевых элементов: 0.5156593918800354
Максимальное значение ненулевых элементов: 1.0
Минимальное значение ненулевых элементов: 0.0


In [36]:
# разобьём наблюдения на тестовую и обучающую выборки
np.random.seed(0)
test_indices = np.random.choice(
    range(user_item_matrix.nnz),
    replace=False,
    size=int(user_item_matrix.nnz * 0.2)
).tolist()
train_data = user_item_matrix.copy()
train_data.data[test_indices] = 0
train_data.eliminate_zeros()
print("размер обучающей выборки: {}".format(train_data.nnz))
test_data = user_item_matrix.copy()
test_data.data[:] = 0
test_data.data[test_indices] = user_item_matrix.data[test_indices]
test_data.eliminate_zeros()
print("размер тестовой выборки: {}".format(test_data.nnz))

размер обучающей выборки: 41289
размер тестовой выборки: 10279


In [37]:
# обучаем  модель
model = LightFM( 
    no_components=10,
    k=5, n=10, 
    loss='bpr', 
    learning_rate=0.05, 
    rho=0.95, epsilon=1e-06, 
    item_alpha=0.0, user_alpha=0.0, 
    max_sampled=10, random_state=1
)

model.fit(train_data, num_threads=1)

train_auc = auc_score(model, train_data).mean()
test_auc = auc_score(model, test_data).mean()
print('auc_score: train %.2f, test %.2f.' % (train_auc, test_auc))

train_recall = recall_at_k(model, train_data).mean()
test_recall = recall_at_k(model, test_data).mean()
print('recall_at_k: train %.2f, test %.2f.' % (train_recall, test_recall))

train_mrr = reciprocal_rank(model, train_data).mean()
test_mrr = reciprocal_rank(model, test_data).mean()
print('MRR: train %.2f, test %.2f.' % (train_mrr, test_mrr))
for k in [5, 10, 15, 20]:
    train_precision = precision_at_k(model, train_data, k=10).mean()
    test_precision = precision_at_k(model, test_data, k=10).mean()
    print('Precision@%d: train %.2f, test %.2f.' % (k, train_precision, test_precision))

auc_score: train 0.64, test 0.63.
recall_at_k: train 0.07, test 0.06.
MRR: train 0.47, test 0.18.
Precision@5: train 0.25, test 0.06.
Precision@10: train 0.25, test 0.06.
Precision@15: train 0.25, test 0.06.
Precision@20: train 0.25, test 0.06.


In [43]:
ratings.head()

,userId,movieId,rating,timestamp,movie_id,user_id
0,1,31,2.5,1260759144,30,0
1,1,1029,3.0,1260759179,833,0
2,1,1061,3.0,1260759182,859,0
3,1,1129,2.0,1260759185,906,0
4,1,1172,4.0,1260759205,931,0


In [41]:
# выберем информацию по жанрам фильмов
movies_tag = ratings[["movieId", "movie_id"]].drop_duplicates().join(
    tags,
    on="movieId",
    rsuffix="codes",
    lsuffix="movies",
    sort=True
)
movies_tag["tags_set"] = movies_tag["tag"]
movies_tag.head()

,movieId,movieIdmovies,movie_idmovies,userId,movieIdcodes,tag,timestamp,movie_idcodes,tag_id,tags_set
495,1,1,0,15.0,1955.0,dentist,1.193435e+09,104.0,252.0,dentist
963,2,2,1,15.0,7478.0,Cambodia,1.170561e+09,301.0,22.0,Cambodia
351,3,3,2,15.0,32892.0,Russian,1.170626e+09,353.0,121.0,Russian
3108,4,4,3,15.0,34162.0,forgettable,1.141392e+09,355.0,296.0,forgettable
964,5,5,4,15.0,35957.0,short,1.141392e+09,360.0,482.0,short


In [42]:
# все возможные теги
from functools import reduce

reduce(lambda acc, ele: acc.union(ele), movies_tag['tags_set'].tolist(), set())

TypeError: 'float' object is not iterable

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# приравняем None и (no genres listed)
movies_tag.loc[movies_tag["tag"] == "(no tags listed)", "tag"] = "None"
# уберём все спецсимволы
movies_tag["tag"] = movies_tag["tag"].apply(
    lambda x: x.replace("-", "")
)

movies_features = CountVectorizer().fit_transform(movies_tag["tag"])
movies_features

In [ ]:
sparse_info(test_data)

In [ ]:
print(movies_tag["tag"][:1])
print(movies_features[0].todense())

In [ ]:
# добавляем к movie_id ещё и информацию о тегах
from scipy.sparse import hstack, identity

features = hstack([
    identity(movies_tag.shape[0]),
    movies_features
])

In [ ]:
features

In [ ]:
hybrid = LightFM( 
    no_components=10,
    k=5, n=10, 
    loss='bpr', 
    learning_rate=0.05, 
    rho=0.95, epsilon=1e-06, 
    item_alpha=0.0, user_alpha=0.0, 
    max_sampled=10, random_state=1
)

hybrid.fit(train_data, item_features = features, num_threads=1)

train_auc = auc_score(hybrid, train_data, item_features=features).mean()
test_auc = auc_score(hybrid, test_data, item_features=features).mean()
print('auc_score: train %.2f, test %.2f.' % (train_auc, test_auc))

train_recall = recall_at_k(hybrid, train_data, item_features=features).mean()
test_recall = recall_at_k(hybrid, test_data, item_features=features).mean()
print('recall_at_k: train %.2f, test %.2f.' % (train_recall, test_recall))

train_mrr = reciprocal_rank(hybrid, train_data, item_features=features).mean()
test_mrr = reciprocal_rank(hybrid, test_data, item_features=features).mean()
print('MRR: train %.2f, test %.2f.' % (train_mrr, test_mrr))
for k in [5, 10, 15, 20]:
    train_precision = precision_at_k(hybrid, train_data, k=10, item_features=features).mean()
    test_precision = precision_at_k(hybrid, test_data, k=10, item_features=features).mean()
    print('Precision@%d: train %.2f, test %.2f.' % (k, train_precision, test_precision))